In [4]:
import os
import gradio as gr
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI


In [6]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
ollama_api_key = os.getenv('OLLAMA_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:2]}")
else:
    print("Groq API Key not set (and this is optional)")
    
if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:2]}")
else:
    print("OpenRouter API Key not set (and this is optional)")

if ollama_api_key:
    print(f"ollama API Key exists and begins {ollama_api_key[:2]}")
else:
    print("ollama API Key not set (and this is optional)")





OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
Groq API Key exists and begins gs
OpenRouter API Key exists and begins sk
ollama API Key exists and begins ad


In [9]:
openai = OpenAI()

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"
openrouter_url = "https://openrouter.ai/api/v1"


gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)


In [10]:
models = ["gpt-4.1-mini", "gemini-2.5-flash", "deepseek-v3.1:671b-cloud", "groq/compound", "openrouter/free" , "google/gemma-3-27b-it:free" ]

clients = {"gpt-4.1-mini": openai, "gemini-2.5-flash": gemini, "deepseek-v3.1:671b-cloud": ollama, "groq/compound": groq, "openrouter/free": openrouter, "openrouter/free": openrouter}

In [11]:
from system_info import retrieve_system_info, rust_toolchain_info

system_info = retrieve_system_info()
rust_info = rust_toolchain_info()
rust_info

{'installed': True,
 'rustc': {'path': 'C:\\Users\\vasanth\\.cargo\\bin\\rustc.EXE',
  'version': 'rustc 1.88.0 (6b00bc388 2025-06-23)',
  'host_triple': 'x86_64-pc-windows-msvc',
  'release': '1.88.0',
  'commit_hash': '6b00bc3880198600130e1cf62b8f8a93494488cc'},
 'cargo': {'path': 'C:\\Users\\vasanth\\.cargo\\bin\\cargo.EXE',
  'version': 'cargo 1.88.0 (873a06493 2025-05-10)'},
 'rustup': {'path': 'C:\\Users\\vasanth\\.cargo\\bin\\rustup.EXE',
  'version': 'rustup 1.28.2 (e4f3ad6f8 2025-04-28)',
  'active_toolchain': 'stable-x86_64-pc-windows-msvc (default)',
  'default_toolchain': '',
  'toolchains': ['stable-x86_64-pc-windows-msvc (active, default)'],
  'targets_installed': ['x86_64-pc-windows-msvc']},
 'rust_analyzer': {'path': 'C:\\Users\\vasanth\\.cargo\\bin\\rust-analyzer.EXE'},
 'env': {'CARGO_HOME': 'C:\\Users\\vasanth\\.cargo',
  'RUSTUP_HOME': 'C:\\Users\\vasanth\\.rustup',
  'RUSTFLAGS': '',
  'CARGO_BUILD_TARGET': ''},
 'execution_examples': ['"C:\\Users\\vasanth\\.cargo\

In [12]:
message = f"""
Below is detailed system information for my machine.

My goal is to compile and run a **single Rust source file (`main.rs`)** with the **fastest possible runtime performance** on this platform. Compile time is not important.

Tasks:
1. Determine whether a Rust toolchain is already installed and sufficient.
2. If not installed, explain whether I need to install one and provide the **simplest step-by-step installation instructions**.
3. If Rust is already set up, provide the **exact shell commands** needed to:
   - Compile `main.rs` with **maximum runtime optimization**
   - Execute the resulting binary

Then, show how to run those commands from Python using `subprocess.run`, filling in **exact values** for:
- `compile_command`
- `run_command`

Constraints:
- Assume this is a single-file Rust program (no Cargo project).
- Prefer `rustc` directly over Cargo unless Cargo is strictly required.
- Optimize for **fastest possible runtime performance** for this platform (e.g., `-O`, `-C target-cpu=native`, LTO, etc.).
- The output should be only the required commands, in **Markdown code blocks**, with no unnecessary commentary.

System information:
{system_info}

Rust toolchain information:
{rust_info}
"""


In [14]:
response = openai.chat.completions.create(model=models[0], messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))

```bash
"C:\Users\vasanth\.cargo\bin\rustc.EXE" main.rs -C opt-level=3 -C target-cpu=native -C lto -o main.exe
.\main.exe
```

```python
import subprocess

compile_command = [
    r"C:\Users\vasanth\.cargo\bin\rustc.EXE",
    "main.rs",
    "-C", "opt-level=3",
    "-C", "target-cpu=native",
    "-C", "lto",
    "-o", "main.exe"
]

run_command = [r".\main.exe"]

subprocess.run(compile_command, check=True)
subprocess.run(run_command, check=True)
```

In [24]:
import subprocess
import io
import sys

compile_command = [
    r"C:\Users\vasanth\.cargo\bin\rustc.EXE",
    "main.rs",
    "-C", "opt-level=3",
    "-C", "target-cpu=native",
    "-C", "lto",
    "-o", "main.exe"
]
run_command = [r".\main.exe"]


In [19]:
language = "Rust" # or "C++"
extension = "rs" if language == "Rust" else "cpp"

system_prompt = f"""
Your task is to convert Python code into high performance {language} code.
Respond only with {language} code. Do not provide any explanation other than occasional comments.
The {language} response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to {language} with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.{language} and then compiled and executed; the compilation command is:
{compile_command}
Respond only with {language} code.
Python code to port:

```python
{python}
```
"""

In [20]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [21]:
def write_output(code):
    with open(f"main.{extension}", "w") as f:
        f.write(code)

In [22]:
def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```rust','').replace('```','')
    return reply

In [25]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [26]:
python_hard = """# Be careful to support large numbers

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [28]:
# Use the commands from GPT 5

def compile_and_run(code):
    write_output(code)
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [33]:
from styles import CSS

with gr.Blocks(title=f"Port from Python to {language}") as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python (original)",
                value=python_hard,
                language="python",
                lines=26
            )
        with gr.Column(scale=6):
            cpp = gr.Code(
                label=f"{language} (generated)",
                value="",
                language="cpp",
                lines=26
            )

    with gr.Row(elem_classes=["controls"]):
        python_run = gr.Button("Run Python", elem_classes=["run-btn", "py"])
        model = gr.Dropdown(models, value=models[0], show_label=False)
        convert = gr.Button(f"Port to {language}", elem_classes=["convert-btn"])
        cpp_run = gr.Button(f"Run {language}", elem_classes=["run-btn", "cpp"])

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(label="Python result", lines=8, elem_classes=["py-out"])
        with gr.Column(scale=6):
            cpp_out = gr.TextArea(label=f"{language} result", lines=8, elem_classes=["cpp-out"])

    convert.click(fn=port, inputs=[model, python], outputs=[cpp])
    python_run.click(fn=run_python, inputs=[python], outputs=[python_out])
    cpp_run.click(fn=compile_and_run, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True, css=CSS, theme=gr.themes.Monochrome())


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "f:\LLM-Engineering-New\.venv\Lib\site-packages\gradio\queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "f:\LLM-Engineering-New\.venv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "f:\LLM-Engineering-New\.venv\Lib\site-packages\gradio\blocks.py", line 2106, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "f:\LLM-Engineering-New\.venv\Lib\site-packages\gradio\blocks.py", line 1588, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "f:\LLM-Engineering-New\.venv\Lib\site-packages\anyio\to_thread.py", line 61, in run_sync
    return await get_async_backend().run_sync

gemini :
    Total Maximum Subarray Sum (20 runs): 10980
    Execution Time: 1.026621 seconds